In [3]:
#import fastText
from scipy.sparse import hstack
from scipy import sparse 
import pandas as pd, numpy as np
#import xgboost
#import gensim
#from gensim.models.wrappers import FastText
from sklearn.metrics import make_scorer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import scipy.sparse as sps
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import cross_val_score , GroupKFold, StratifiedKFold
import re
#from xgboost import XGBRegressor
import string
#import pymorphy2

Yandex ML track 
https://contest.yandex.ru/algorithm2018/contest/7914 
Подбор реплик для продолжения диалога

In [2]:
fastText

<module 'fastText' from '/usr/local/lib/python3.6/dist-packages/fastText/__init__.py'>

In [2]:
ft_model = fastText.load_model("data/model.bin")
#ft_model_en = fastText.load_model("data/model-en.bin")

In [3]:
train = pd.read_csv('data/train.tsv', sep='\t', quotechar=' ', header = None)
train.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']
test = pd.read_csv('data/public.tsv', sep='\t', quotechar = ' ', header = None)
test.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']
final = pd.read_csv('data/final.tsv', sep='\t', quotechar = ' ', header = None)
final.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

In [4]:
def label_enc(x ,reverse = False):
    if reverse == False:
        if x == 'bad':
            return 0
        elif x == 'neutral':
            return 1
        else:
            return 2
    else:
        if x == 0:
            return 'bad'
        elif x == 1:
            return 'neutral'
        else:
            return 'good'
        
def get_context_number(row):
    if pd.isnull(row['context_1']):
        return 1
    elif pd.isnull(row['context_2']):
        return 2
    else:
        return 3
        
        
        
train['label'] = train['label'].apply(lambda x: label_enc(x))

train['context_number'] = train.apply(lambda x: get_context_number(x), axis = 1)
test['context_number'] = test.apply(lambda x: get_context_number(x), axis = 1)
    
train.fillna('', inplace=True)
test.fillna('', inplace=True)

final['context_number'] = final.apply(lambda x: get_context_number(x), axis = 1)
final.fillna('', inplace=True)

In [5]:
train.fillna('', inplace=True)
test.fillna('', inplace=True)
final.fillna('', inplace=True)

In [5]:
def drop_punctuation(x):
    '''
        x : string
            String from which the punctuation is removed
    '''
    return re.sub('[%s]' % string.punctuation, ' ', x)

train['reply_without_punct'] = train['reply'].apply(lambda x: drop_punctuation(x))
train['reply_without_punct'] = train['reply_without_punct'].apply(lambda x: ' '.join(x.split()))
train['reply_arr'] = train['reply_without_punct'].str.split(" ").tolist()


test['reply_without_punct'] = test['reply'].apply(lambda x: drop_punctuation(x))
test['reply_without_punct'] = test['reply_without_punct'].apply(lambda x: ' '.join(x.split()))
test['reply_arr'] = test['reply_without_punct'].str.split(" ").tolist()

morph = pymorphy2.MorphAnalyzer()


def get_normal_form(row):
    s = ''
    for x in row:
        p = morph.parse(x)[0]
        s += p.normal_form + ' '
    return s

def check_if_verb(row):
    f = 0
    for x in row:
        p = morph.parse(x)[0]
        if 'VERB' in p.tag:
            f =1
            
    return f

def clear(row):
    s = ''
    for x in row:
        p = morph.parse(x)[0]
        if ( 'LATN' in p.tag) and ('PNCT'  in p.tag) and ('NUMB'  in p.tag) and ('intg' in p.tag) and ('real'  in p.tag) and ('ROMN'  in p.tag) and ( 'UNKN'  in p.tag):
            s += x + ' '

train['normal_reply'] = train['reply_arr'].apply(lambda x: get_normal_form(x))
    
train['reply'] = train['reply_arr'].apply(lambda x: clear(x))
test['reply'] = test['reply_arr'].apply(lambda x: clear(x))

train['normal_reply'] = train['reply_arr'].apply(lambda x: get_normal_form(x))

#train['reply_contains_verb'] = train['reply_arr'].apply(lambda x: check_if_verb(x))

In [6]:
final['reply_without_punct'] = final['reply'].apply(lambda x: drop_punctuation(x))
final['reply_without_punct'] = final['reply_without_punct'].apply(lambda x: ' '.join(x.split()))
final['reply_arr'] = final['reply_without_punct'].str.split(" ").tolist()

In [8]:
def check_if_verb(row):
    f = 0
    for x in row:
        p = morph.parse(x)[0]
        if 'VERB' in p.tag:
            f =1
            
    return f

def check_if_noun(row):
    f = 0
    for x in row:
        p = morph.parse(x)[0]
        if 'NOUN' in p.tag:
            f =1
            
    return f

def check_if_advb(row):
    f = 0
    for x in row:
        p = morph.parse(x)[0]
        if 'ADVB' in p.tag:
            f =1
            
    return f 

def check_if_numr(row):
    f = 0
    for x in row:
        p = morph.parse(x)[0]
        if 'NUMR' in p.tag:
            f =1
            
    return f

train['reply_contains_verb'] = train['reply_arr'].apply(lambda x: check_if_verb(x))
test['reply_contains_verb'] = test['reply_arr'].apply(lambda x: check_if_verb(x))

train['reply_contains_noun'] = train['reply_arr'].apply(lambda x: check_if_noun(x))
test['reply_contains_noun'] = test['reply_arr'].apply(lambda x: check_if_noun(x))

train['reply_contains_advb'] = train['reply_arr'].apply(lambda x: check_if_advb(x))
test['reply_contains_advb'] = test['reply_arr'].apply(lambda x: check_if_advb(x))

train['reply_contains_numr'] = train['reply_arr'].apply(lambda x: check_if_numr(x))
test['reply_contains_numr'] = test['reply_arr'].apply(lambda x: check_if_numr(x))

train.head()

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence,context_number,reply_without_punct,reply_arr,normal_reply,reply_contains_verb,reply_contains_noun,reply_contains_advb,reply_contains_numr
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",0,не могу .,2,0.875352,3,не могу,"[не, могу]",не мочь,1,0,0,0
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",1,"нет , звонить буду я .",1,0.900968,3,нет звонить буду я,"[нет, звонить, буду, я]",нет звонить быть я,1,0,0,0
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",2,"слушай , я не мог уйти .",0,0.884320,3,слушай я не мог уйти,"[слушай, я, не, мог, уйти]",слушай я не мочь уйти,1,0,0,0
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",3,я не прекращу звонить .,2,0.982530,3,я не прекращу звонить,"[я, не, прекращу, звонить]",я не прекратить звонить,1,0,0,0
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу .","ладно , повесь трубку .",4,я звоню им .,2,0.838054,3,я звоню им,"[я, звоню, им]",я звонить имя,1,1,0,0


In [7]:
train['context_0_without_punct'] = train['context_0'].apply(lambda x: drop_punctuation(x))
train['context_0_without_punct'] = train['context_0_without_punct'].apply(lambda x: ' '.join(x.split()))
train['context_0_arr'] = train['context_0_without_punct'].str.split(" ").tolist()


test['context_0_without_punct'] = test['context_0'].apply(lambda x: drop_punctuation(x))
test['context_0_without_punct'] = test['context_0_without_punct'].apply(lambda x: ' '.join(x.split()))
test['context_0_arr'] = test['context_0_without_punct'].str.split(" ").tolist()


train['context_1_without_punct'] = train['context_1'].apply(lambda x: drop_punctuation(x))
train['context_1_without_punct'] = train['context_1_without_punct'].apply(lambda x: ' '.join(x.split()))
train['context_1_arr'] = train['context_1_without_punct'].str.split(" ").tolist()


test['context_1_without_punct'] = test['context_1'].apply(lambda x: drop_punctuation(x))
test['context_1_without_punct'] = test['context_1_without_punct'].apply(lambda x: ' '.join(x.split()))
test['context_1_arr'] = test['context_1_without_punct'].str.split(" ").tolist()

train['context_2_without_punct'] = train['context_2'].apply(lambda x: drop_punctuation(x))
train['context_2_without_punct'] = train['context_2_without_punct'].apply(lambda x: ' '.join(x.split()))
train['context_2_arr'] = train['context_2_without_punct'].str.split(" ").tolist()


test['context_2_without_punct'] = test['context_2'].apply(lambda x: drop_punctuation(x))
test['context_2_without_punct'] = test['context_2_without_punct'].apply(lambda x: ' '.join(x.split()))
test['context_2_arr'] = test['context_2_without_punct'].str.split(" ").tolist()

def get_gender(row):
    is_masc = 0
    is_femn = 0
    is_neut = 0
    for x in row:
        p = morph.parse(x)[0]
        if p.tag.gender == 'masc':
            is_masc = 1
        elif p.tag.gender == 'femn':
            is_femn = 1
        elif p.tag.gender == 'neut':
            is_neut = 1  
    if is_masc:
        return 1
    if is_femn:
        return 2
    if is_neut:
        return 3
    return 0


"train['reply_gender'] = train['reply_arr'].apply(lambda x: get_gender(x))\ntest['reply_gender'] = test['reply_arr'].apply(lambda x: check_if_verb(x))\n\ntrain['context_0_gender'] = train['context_0_arr'].apply(lambda x: get_gender(x))\ntest['context_0_gender'] = test['context_0_arr'].apply(lambda x: check_if_verb(x))\n\ntrain['context_1_gender'] = train['context_1_arr'].apply(lambda x: get_gender(x))\ntest['context_1_gender'] = test['context_1_arr'].apply(lambda x: check_if_verb(x))\n\ntrain['context_2_gender'] = train['context_2_arr'].apply(lambda x: get_gender(x))\ntest['context_2_gender'] = test['context_2_arr'].apply(lambda x: check_if_verb(x))"

In [8]:
final['context_0_without_punct'] = final['context_0'].apply(lambda x: drop_punctuation(x))
final['context_0_without_punct'] = final['context_0_without_punct'].apply(lambda x: ' '.join(x.split()))
final['context_0_arr'] = final['context_0_without_punct'].str.split(" ").tolist()

final['context_1_without_punct'] = final['context_1'].apply(lambda x: drop_punctuation(x))
final['context_1_without_punct'] = final['context_1_without_punct'].apply(lambda x: ' '.join(x.split()))
final['context_1_arr'] = final['context_1_without_punct'].str.split(" ").tolist()

final['context_2_without_punct'] = final['context_2'].apply(lambda x: drop_punctuation(x))
final['context_2_without_punct'] = final['context_2_without_punct'].apply(lambda x: ' '.join(x.split()))
final['context_2_arr'] = final['context_2_without_punct'].str.split(" ").tolist()



In [9]:
def drop_punctuation(x):
    '''
        x : string
            String from which the punctuation is removed
    '''
    return re.sub('[%s]' % string.punctuation, ' ', x)

def get_normal_form(row):
    s = ''
    for x in row:
        p = morph.parse(x)[0]
        s += p.normal_form + ' '
    return s

morph = pymorphy2.MorphAnalyzer()


train['text'] = train['context_0'] + " " + train['context_1'] + " " + train['context_2'] 
test['text'] = test['context_0'] + " " + test['context_1'] + " " + test['context_2'] 

train['text_without_punct'] = train['text'].apply(lambda x: drop_punctuation(x))
train['text_without_punct'] = train['text_without_punct'].apply(lambda x: ' '.join(x.split()))
train['text_arr'] = train['text_without_punct'].str.split(" ").tolist()


test['text_without_punct'] = test['text'].apply(lambda x: drop_punctuation(x))
test['text_without_punct'] = test['text_without_punct'].apply(lambda x: ' '.join(x.split()))
test['text_arr'] = test['text_without_punct'].str.split(" ").tolist()

train['normal_text'] = train['text_arr'].apply(lambda x: get_normal_form(x))

In [10]:
final['text'] = final['context_0'] + " " + final['context_1'] + " " + final['context_2'] 


final['text_without_punct'] = final['text'].apply(lambda x: drop_punctuation(x))
final['text_without_punct'] = final['text_without_punct'].apply(lambda x: ' '.join(x.split()))
final['text_arr'] = final['text_without_punct'].str.split(" ").tolist()


In [11]:
train['normal_context_0'] = train['context_0_arr'].apply(lambda x: get_normal_form(x))
train['normal_context_1'] = train['context_1_arr'].apply(lambda x: get_normal_form(x))
train['normal_context_2'] = train['context_2_arr'].apply(lambda x: get_normal_form(x))

test['normal_context_0'] = test['context_0_arr'].apply(lambda x: get_normal_form(x))
test['normal_context_1'] = test['context_1_arr'].apply(lambda x: get_normal_form(x))
test['normal_context_2'] = test['context_2_arr'].apply(lambda x: get_normal_form(x))

In [12]:
final['normal_context_0'] = final['context_0_arr'].apply(lambda x: get_normal_form(x))
final['normal_context_1'] = final['context_1_arr'].apply(lambda x: get_normal_form(x))
final['normal_context_2'] = final['context_2_arr'].apply(lambda x: get_normal_form(x))

In [13]:
test['normal_reply'] = test['reply_arr'].apply(lambda x: get_normal_form(x))
test['normal_text'] = test['text_arr'].apply(lambda x: get_normal_form(x))

In [14]:
final['normal_reply'] = final['reply_arr'].apply(lambda x: get_normal_form(x))
final['normal_text'] = final['text_arr'].apply(lambda x: get_normal_form(x))

In [17]:
def get_case(row):
    is_nomn = 0
    is_gent = 0
    is_datv = 0
    is_accs = 0
    is_ablt= 0
    is_loct = 0
    is_voct = 0
    is_gen2= 0
    is_acc2 = 0
    is_loc2 = 0
    for x in row:
        p = morph.parse(x)[0]
        if p.tag.case == 'nomn':
            is_nomn = 1
        elif p.tag.case == 'gent':
            is_gent = 1
        elif p.tag.case == 'datv':
            is_datv = 1  
        elif p.tag.case == 'accs':
            is_accs = 1
        elif p.tag.case == 'ablt':
            is_ablt = 1 
        elif p.tag.case == 'loct':
            is_loct = 1
        elif p.tag.case == 'voct':
            is_voct = 1  
        elif p.tag.case == 'gen2':
            is_gen2 = 1
        elif p.tag.case == 'acc2':
            is_acc2 = 1  
        elif p.tag.case == 'loc2':
            is_loc2 = 1  
    if is_nomn:
        return 1
    if is_gent:
        return 2
    if is_datv:
        return 3
    if is_accs:
        return 4
    if is_ablt:
        return 5
    if is_loct:
        return 6
    if is_voct:
        return 7
    if is_gen2:
        return 8
    if is_acc2:
        return 9
    if is_loc2:
        return 10
    return 0

train['reply_case'] = train['reply_arr'].apply(lambda x: get_case(x))
test['reply_case'] = test['reply_arr'].apply(lambda x: get_case(x))

train['context_0_case'] = train['context_0_arr'].apply(lambda x: get_case(x))
test['context_0_case'] = test['context_0_arr'].apply(lambda x: get_case(x))

train['context_1_case'] = train['context_1_arr'].apply(lambda x: get_gender(x))
test['context_1_case'] = test['context_1_arr'].apply(lambda x: get_case(x))

train['context_2_case'] = train['context_2_arr'].apply(lambda x: get_case(x))
test['context_2_case'] = test['context_2_arr'].apply(lambda x: get_case(x))

In [18]:
def get_normal_text_without_prs(row):
    s = ''
    for x in row:
        p = morph.parse(x)[0]        
        if ('PREP' not in p.tag) and ('CONJ' not in p.tag) and ('PRCL' not in p.tag) and ('INTJ' not in p.tag):
            s += p.normal_form + ' '
    return s

train['normal_reply_without_prs'] = train['reply_arr'].apply(lambda x: get_normal_text_without_prs(x))
train['normal_text_without_prs'] = train['text_arr'].apply(lambda x: get_normal_text_without_prs(x))
test['normal_reply_without_prs'] = test['reply_arr'].apply(lambda x: get_normal_text_without_prs(x))
test['normal_text_without_prs'] = test['text_arr'].apply(lambda x: get_normal_text_without_prs(x))

In [19]:
final['normal_reply_without_prs'] = final['reply_arr'].apply(lambda x: get_normal_text_without_prs(x))
final['normal_text_without_prs'] = final['text_arr'].apply(lambda x: get_normal_text_without_prs(x))

In [20]:
def get_text_without_prs(row):
    s = ''
    for x in row:
        p = morph.parse(x)[0]        
        if ('PREP' not in p.tag) and ('CONJ' not in p.tag) and ('PRCL' not in p.tag) and ('INTJ' not in p.tag):
            s += x+ ' '
    return s

train['reply_without_prs'] = train['reply_arr'].apply(lambda x: get_text_without_prs(x))
train['text_without_prs'] = train['text_arr'].apply(lambda x: get_text_without_prs(x))
test['reply_without_prs'] = test['reply_arr'].apply(lambda x: get_text_without_prs(x))
test['text_without_prs'] = test['text_arr'].apply(lambda x: get_text_without_prs(x))

In [21]:
final['reply_without_prs'] = final['reply_arr'].apply(lambda x: get_text_without_prs(x))
final['text_without_prs'] = final['text_arr'].apply(lambda x: get_text_without_prs(x))

In [15]:
def Vect(df, test):
        tfidf = TfidfVectorizer()

        tfidf.fit(dt['normal_text'])

        context = tfidf.transform(df['normal_text']) 
        context_t = tfidf.transform(test['normal_text'])

        tfidf.fit(dt['normal_reply'])

        reply = tfidf.fit_transform(df['normal_reply'])
        reply_t = tfidf.transform(test['normal_reply'])

        return sps.hstack((context, reply)), \
               sps.hstack((context_t, reply_t))
        

        
def Vect5(train, test):
        t1_ft = np.vstack(train['context_0_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        t2_ft = np.vstack(train['context_1_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        t3_ft = np.vstack(train['context_2_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        t5_ft = np.vstack(train['reply_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))

        te1_ft = np.vstack(test['context_0_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        te2_ft = np.vstack(test['context_1_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        te3_ft = np.vstack(test['context_2_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))
        te5_ft = np.vstack(test['reply_without_punct'].apply(lambda x: ft_model.get_sentence_vector(x)))

        return np.hstack([t1_ft, t2_ft, t3_ft, t5_ft]), \
                np.hstack([te1_ft, te2_ft, te3_ft, te5_ft])



dt = pd.concat([train, test])
X_train, X_test = Vect(train, final)
X_train5, X_test5 = Vect5(train, final)
X_train = hstack((sparse.csr_matrix(X_train), sparse.csr_matrix(X_train5)))
X_test = hstack((sparse.csr_matrix(X_test), sparse.csr_matrix(X_test5)))


"X_train = hstack((X_train, np.array(train['reply_contains_verb'])[:,None]))\nX_train = hstack((X_train, np.array(train['reply_contains_noun'])[:,None]))\nX_test = hstack((X_test, np.array(test['reply_contains_verb'])[:,None]))\nX_test = hstack((X_test, np.array(test['reply_contains_noun'])[:,None]))\n\nX_train = hstack((X_train, np.array(train['reply_contains_advb'])[:,None]))\nX_train = hstack((X_train, np.array(train['reply_contains_numr'])[:,None]))\nX_test = hstack((X_test, np.array(test['reply_contains_advb'])[:,None]))\nX_test = hstack((X_test, np.array(test['reply_contains_numr'])[:,None]))"

In [ ]:
train['target'] = train[0:50000]['label'] * train[0:50000]['confidence']
clf = LGBMRegressor().fit(X_train, train['target'])
y_test = clf.predict(X_test)
final['target']  = -y_test
final.sort_values(by=['context_id', 'target'])[['context_id', 'reply_id']].to_csv('subm.csv', encoding='utf-8', sep=' ', index=False)